In [1]:
%cd /home/tkm290/repos/erc

/home/tkm290/repos/erc


In [2]:
from fairseq.models.roberta import RobertaModel
import torch
import os
import torch
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from glob import glob
from fairseq.data.data_utils import collate_tokens

In [3]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

Using cache found in /home/tkm290/.cache/torch/hub/pytorch_fairseq_master


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): FusedLayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): FusedLayerNorm(torch.Size([1024]), eps=1e-05, elementwise_

In [3]:
def evalute_SPLIT(roberta, DATASET, num_utt, batch_size, SPLIT):

    def label_fn(label):
        return roberta.task.label_dictionary.string(
            [label + roberta.task.label_dictionary.nspecial])

    y_true = []
    y_pred = []

    X = {}
    for i in range(num_utt):
        X[i] = os.path.join(DATASET_DIR, DATASET,
                            'roberta', SPLIT + f'.input{i}')

        with open(X[i], 'r') as stream:
            X[i] = [line.strip() for line in stream.readlines()]

    Y = os.path.join(DATASET_DIR, DATASET, 'roberta', SPLIT + '.label')
    with open(Y, 'r') as stream:
        Y = [line.strip() for line in stream.readlines()]
    
    for i in range(num_utt):
        assert len(X[i]) == len(Y)
    
    original_length = len(Y)
    num_batches = original_length // batch_size
    
    for i in range(num_utt):
        X[i] = [X[i][j*batch_size:(j+1)*batch_size] for j in range(num_batches)] + \
        [[X[i][j]] for j in range(batch_size*num_batches, original_length)]

    Y = [Y[j*batch_size:(j+1)*batch_size] for j in range(num_batches)] + \
    [[Y[j]] for j in range(batch_size*num_batches, original_length)]
    
    
    for i in range(num_utt):
        assert len(X[i]) == len(Y)    
    
    for idx in tqdm(range(len(Y))):
        batch = [X[i][idx] for i in range(num_utt)]
        batch = list(map(list, zip(*batch)))

        batch = collate_tokens(
            [roberta.encode(*[sentence for sentence in sentences]) for sentences in batch], pad_idx=1
        )

        logprobs = roberta.predict(DATASET + '_head', batch)
        pred = logprobs.argmax(dim=1)
        label = Y[idx]
            
        assert len(pred) == len(label)
        for p, l in zip(pred, label):
            y_true.append(l)
            y_pred.append(label_fn(p))
        
    
    assert original_length == len(y_true) == len(y_pred), \
    f"{original_length}, {len(y_true)}, {len(y_pred)}"

    scores_all = {}
    scores_all['f1_weighted'] = f1_score(y_true, y_pred, average='weighted')
    scores_all['f1_micro'] = f1_score(y_true, y_pred, average='micro')
    scores_all['f1_macro'] = f1_score(y_true, y_pred, average='macro')

    return scores_all

In [10]:
os.rename?

In [4]:
DATASET='MELD'
checkpoint_dir='checkpoints_20210309_09391615282793'
BASE_DIR='models/roberta.large/MELD/20210309_09391615282793'
num_utt=2
METRIC='f1_weighted'
DATASET_DIR = "Datasets/"
MODEL_DIR = 'models/'
DATASETS_SUPPORTED = ['MELD', 'IEMOCAP',
                      'CAER', 'EmoryNLP', 'DailyDialog']
batch_size=8
use_cuda=True


stats = {}
for model_path in tqdm(glob(os.path.join(checkpoint_dir, '*.pt'))):
    checkpoint_file = os.path.basename(model_path)
    roberta = RobertaModel.from_pretrained(
        checkpoint_dir,
        checkpoint_file=checkpoint_file,
        data_name_or_path=os.path.join(DATASET_DIR, DATASET, 'roberta/bin')
    )

    roberta.eval()  # disable dropout
    if use_cuda:
        roberta.cuda()

    scores = evalute_SPLIT(roberta, DATASET, num_utt, batch_size, SPLIT='val')
    print(model_path, scores)
    stats[checkpoint_file] = scores
    del roberta

print(stats)


checkpoints_20210309_09391615282793/checkpoint7.pt {'f1_weighted': 0.6309204767822929, 'f1_micro': 0.6384129846708747, 'f1_macro': 0.5063188012350345}



checkpoints_20210309_09391615282793/checkpoint_best.pt {'f1_weighted': 0.6183968579443475, 'f1_micro': 0.6447249774571686, 'f1_macro': 0.4503307820913872}



checkpoints_20210309_09391615282793/checkpoint5.pt {'f1_weighted': 0.6231087049601733, 'f1_micro': 0.6330027051397655, 'f1_macro': 0.49152458994388437}



checkpoints_20210309_09391615282793/checkpoint6.pt {'f1_weighted': 0.6442592627635866, 'f1_micro': 0.654643823264202, 'f1_macro': 0.5119895419140768}



checkpoints_20210309_09391615282793/checkpoint4.pt {'f1_weighted': 0.6156420612063028, 'f1_micro': 0.6447249774571686, 'f1_macro': 0.44624706195446867}



checkpoints_20210309_09391615282793/checkpoint2.pt {'f1_weighted': 0.5968771797353294, 'f1_micro': 0.6230838593327321, 'f1_macro': 0.4097765211967042}



checkpoints_20210309_09391615282793/checkpoint8.pt {'f1_weighted': 0.6351746606290302, 'f1_micro': 0.6384129846708747, 'f1_macro': 0.5143937774768703}



checkpoints_20210309_09391615282793/checkpoint3.pt {'f1_weighted': 0.6183968579443475, 'f1_micro': 0.6447249774571686, 'f1_macro': 0.4503307820913872}



checkpoints_20210309_09391615282793/checkpoint1.pt {'f1_weighted': 0.25454806551872444, 'f1_micro': 0.42470694319206487, 'f1_macro': 0.08692784189695694}



checkpoints_20210309_09391615282793/checkpoint_last.pt {'f1_weighted': 0.6351746606290302, 'f1_micro': 0.6384129846708747, 'f1_macro': 0.5143937774768703}

{'checkpoint7.pt': {'f1_weighted': 0.6309204767822929, 'f1_micro': 0.6384129846708747, 'f1_macro': 0.5063188012350345}, 'checkpoint_best.pt': {'f1_weighted': 0.6183968579443475, 'f1_micro': 0.6447249774571686, 'f1_macro': 0.4503307820913872}, 'checkpoint5.pt': {'f1_weighted': 0.6231087049601733, 'f1_micro': 0.6330027051397655, 'f1_macro': 0.49152458994388437}, 'checkpoint6.pt': {'f1_weighted': 0.6442592627635866, 'f1_micro': 0.654643823264202, 'f1_macro': 0.5119895419140768}, 'checkpoint4.pt': {'f1_weighted': 0.6156420612063028, 'f1_micro': 0.6447249774571686, 'f1_macro': 0.44624706195446867}, 'checkpoint2.pt': {'f1_weighted': 0.5968771797353294, 'f1_micro': 0.6230838593327321, 'f1_macro': 0.4097765211967042}, 'checkpoint8.pt': {'f1_weighted': 0.6351746606290302, 'f1_micro': 0.6384129846708747, 'f1_macro': 0.5143937774768703}, 'che

In [5]:
foo = {key: val['f1_weighted'] for key, val in stats.items()}

In [6]:
foo

{'checkpoint7.pt': 0.6309204767822929,
 'checkpoint_best.pt': 0.6183968579443475,
 'checkpoint5.pt': 0.6231087049601733,
 'checkpoint6.pt': 0.6442592627635866,
 'checkpoint4.pt': 0.6156420612063028,
 'checkpoint2.pt': 0.5968771797353294,
 'checkpoint8.pt': 0.6351746606290302,
 'checkpoint3.pt': 0.6183968579443475,
 'checkpoint1.pt': 0.25454806551872444,
 'checkpoint_last.pt': 0.6351746606290302}

In [7]:
max(foo, key=foo.get)

'checkpoint6.pt'

In [8]:
model_path

'checkpoints_20210309_09391615282793/checkpoint_last.pt'